In [1]:
import os
from crewai import Agent, Task, Crew, Process
from langchain.llms import Ollama
from narrative_llm_agent.agents.narrative import NarrativeAgent
from narrative_llm_agent.agents.job import JobAgent
from narrative_llm_agent.agents.analyst import AnalystAgent
from narrative_llm_agent.agents.workspace import WorkspaceAgent
# from narrative_llm_agent.agents.KnowledgeGraph import KGAgent
from langchain_openai import ChatOpenAI

## Expected environment variables
* KB_AUTH_TOKEN - your ci.kbase.us auth token
* OPENAI_API_KEY - open ai access, if you're using that LLM
* AGENTOPS_API_KEY - if using agentops - bother Bill about this if you want to use it

In [2]:
kbase_token = os.environ["KB_AUTH_TOKEN"]

In [3]:
import agentops
agentops.init()

### If using Ollama-based agents, run `ollama serve` from a terminal first!

In [3]:
# used_llm = Ollama(model="mixtral")
# used_llm = ChatOpenAI(model_name="gpt-4", temperature=0)
used_llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0)
# used_llm = Ollama(model="openhermes")
# used_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
# used_llm = Ollama(model="llama3")

In [4]:
# set up the crew
narr_expert = NarrativeAgent(kbase_token, used_llm)
job_expert = JobAgent(kbase_token, used_llm)
analyst = AnalystAgent(kbase_token, used_llm)
ws_expert = WorkspaceAgent(kbase_token, used_llm)
# kg_er = KGAgent(kbase_token, used_llm)

crew_agents = [analyst.agent, ws_expert.agent, job_expert.agent, narr_expert.agent] #, kg_er.agent]

In [18]:
task1 = Task(description="""
    I have sequenced a new microbe. I have a fastq file of reads from paired end illumina
    sequencing and would like to check its quality. What app should I use? Please suggest only one 
    app at a time for a particular step. If you must suggest alternative apps please 
    limit to only one.
    """, 
    agent=analyst.agent
)

task2 = Task(description="""
    Use the knowledge graph to make sure that the apps that you suggest exist
    After you check its availability in the Knowledge Graph, ask a human if they approve of 
    the suggested app. If not, then check for the additional 
    recommended app in the Knowledge Graph. Return only the app ID. Note that the app ID is not the same as the
    app name, and will have a format of "<module>/<name>" Return only the app ID in that format with no additional text.
    """,
    agent=kg_er.agent
)
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[analyst.agent,kg_er.agent],
  tasks=[task1,task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': '\n    I ...h novel sequence data.)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [6]:
crew.kickoff()

[DEBUG]: Working Agent: Computational Biologist and Geneticist
[INFO]: Starting Task: 
    I have sequenced a new microbe. I have a fastq file of reads from paired end illumina
    sequencing and would like to check its quality. What app should I use? Please suggest only one 
    app at a time for a particular step. If you must suggest alternative apps please 
    limit to only one.
    


> Entering new CrewAgentExecutor chain...
Use Tool: KBase app catalog retrieval tool
Input: I need to find an app that can check the quality of a fastq file from paired end illumina sequencing.initing doc chain with dir /home/ac.wjriehl/ke_llm/narrative_llm_agent/narrative_llm_agent/agents/vector_db_app_catalog
 

{'query': 'I need to find an app that can check the quality of a fastq file from paired end illumina sequencing.', 'chat_history': '', 'result': "Given the additional context provided, which includes a broader range of tools available for sequencing data analysis in a bioinformatics workflo

/home/ac.wjriehl/.cache/pypoetry/virtualenvs/ke-llm-o4_-74U3-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `Information` with `{'entity': 'FastQC', 'entity_type': 'AppCatalog'}`


Need additional information, which of these did you mean: 
{'candidate': 'Join Overlapping Mate Pairs with ea-utils FASTQ-JOIN'}
{'candidate': 'Assess Read Quality with FastQC - v0.12.1'}
{'candidate': 'SINGLE END READS'}I found multiple entries related to FastQC. Could you please specify which one you are interested in?

1. Join Overlapping Mate Pairs with ea-utils FASTQ-JOIN
2. Assess Read Quality with FastQC - v0.12.1
3. SINGLE END READS
I found multiple entries related to FastQC. Could you please specify which one you are interested in?

1. Join Overlapping Mate Pairs with ea-utils FASTQ-JOIN
2. Assess Read Quality with FastQC - v0.12.1
3. SINGLE END READS

> Finished chain.
 

I found multiple entries related to FastQC. Could you please specify which one you are interested in?

1. Join Overlapping Mate Pairs with ea-utils FASTQ-JOIN
2. Assess Read Quality wit

'Assess Read Quality with FastQC - v0.12.1'

In [5]:
# check ws agent
narrative_id = 72233
get_reads_task = Task(
    description=f"""Fetch a list of reads objects from KBase Workspace {narrative_id}. 
    Return only the names of the reads objects. These are objects of type "KBaseFile.PairedEndLibrary". 
    Do not use the "name" field to infer object type. 
    Do not prompt the user for additional information.
    If there are no reads objects, return a message saying so and stop.""",
    agent=analyst.agent,
    extra_content=narrative_id
)

In [10]:
reads_crew = Crew(
    agents=crew_agents,
    tasks=[get_reads_task],
    verbose=2,
    process=Process.sequential
)

2024-03-12 15:41:56,542 - 140225432743936 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [11]:
reads_crew.kickoff()

[DEBUG]: Working Agent: Computational Biologist and Geneticist
[INFO]: Starting Task: Fetch a list of reads objects from KBase Workspace 7998. 
    Return only the names of the reads objects. These are objects of type "KBaseFile.PairedEndLibrary". 
    Do not use the "name" field to infer object type. 
    Do not prompt the user for additional information.
    If there are no reads objects, return a message saying so and stop.


> Entering new CrewAgentExecutor chain...
Use Tool: Delegate work to co-worker
coworker: Workspace Manager
task: Fetch a list of reads objects from KBase Workspace 7998.
context: I need to retrieve a list of reads objects specifically of type "KBaseFile.PairedEndLibrary" from KBase Workspace 7998. It's crucial to identify these objects not by their "name" field but by their actual type to ensure accuracy. The task involves accessing the workspace, filtering objects by their type to find those that match "KBaseFile.PairedEndLibrary", and then compiling a list of

'The names of the objects in KBase Workspace 7998 of type "KBaseFile.PairedEndLibrary" are as follows:\n- Ecoli-Test\n- chr14-longjump\n- chr14-shortjump\n- frags\n- small.interlaced_reads\n- mock2pct\n- mock2pct.250.bbqc.sampled\n- gut-20x.fastq_reads'

In [12]:
job_expert.role

'Job and App Manager'

In [10]:
narrative_id = 74706
reads_id = "74706/2/1"


    # description = """
    # I have a set of paired-end reads stored in KBase that I would like to check for quality.
    # What is the id of the app I should use?
    # The app must be available in KBase.
    # Return only the app id.
    # """,

get_reads_qc_app_task = Task(
    description = """Get the app id for the KBase FastQC app. Return only the app id.""",
    expected_output = "An app id with the format module/app, with a single forward-slash",
    agent=analyst.agent
)

get_app_params_task = Task(
    description = f"""
    From the given KBase app id, fetch the list of parameters needed to run it. Use the App and Job manager agent 
    for assistance. With the knowledge that there is
    a paired-end reads object with id "{reads_id}", populate a dictionary with the parameters where the keys
    are parameter ids, and values are the proper parameter values, or their default values if no value
    can be found or calculated. Return only the dictionary of inputs and the app id for use in the next task. 
    Do not add comments or other text. The dictionary of inputs and the app id must not be combined into a single
    dictionary.
    """,
    expected_output = "A dictionary of parameters used to run the app with the given id.",
    agent=analyst.agent
)

start_job_task = Task(
    description = f"""
    Using the app parameters and app id, use provided tools to start a new KBase app, which will return a job id.
    Use that job id to create a new App Cell in the narrative with id {narrative_id}. The Return only the 
    job id.
    """,
    expected_output = "A KBase job id string.",
    agent=job_expert.agent
)

make_app_cell_task = Task(
    description = f"""
    Using the job id, create an app cell in narrative {narrative_id}. The add_app_cell tool is useful here.
    """,
    expected_output = "A note with either success or failure of saving the new cell",
    agent=narr_expert.agent
)

monitor_job_task = Task(
    description = """
    Using the job id, monitor the progress of the running job. The monitor_job tool is helpful here. 
    If completed, continue to the next task. If in an error state, summarize the error for the user and stop.
    """,
    expected_output = "Return either a note saying that the job has completed, or a summary of the job error.",
    agent=job_expert.agent,
    context=[start_job_task]
)

job_completion_task = Task(
    description="""Use a tool to fetch the status of KBase job with the given job id.
    If it is completed without being an error, retrieve the job results.
    If the job results contain a reference to a report, with an UPA (a string with format number/number/number),
    return it. Do not add any additional text, just return only the UPA. 
    You must always use a tool when interacting with KBase services or databases. If this is delegated, make sure
    the delegated agent uses a tool when interacting with KBase.
    """,
    expected_output = "An UPA of the format 'number/number/number', representing the output of a KBase job, or an error.",
    agent=job_expert.agent,
    context = [start_job_task]
)

report_retrieval_task = Task(
    description="""Using the provided UPA (a string with format number/number/number), 
    use a tool to get the report object from the Workspace service. UPAs are unique permanent addresses used to identify data objects.
    The Workspace Manager will be helpful here. Make sure the delegated agent uses a tool for interacting with KBase.
    Return the full report text. Your final answer MUST be the report text.
    """,
    expected_output = "The text of a KBase app report object",
    agent=ws_expert.agent
)

report_analysis_task = Task(
    description="""Analyze the report text and derive some insight into the result. Write a brief summary, including bullet points 
    when appropriate, formatted in markdown. Your final answer MUST be the summary of the report.""",
    expected_output = "A summary of the report from the previous task",
    agent=analyst.agent
)

save_analysis_task = Task(
    description=f"""Save the analysis by adding a markdown cell to the Narrative with id {narrative_id}. The markdown text must
    be the analysis text. If not successful, say so and stop.""",
    expected_output = "A note with either success or failure of saving the new cell",
    agent=narr_expert.agent,
    extra_content=narrative_id
)

# results_crew = Crew(
#     agents=crew_agents,
#     tasks=[job_completion_task, report_retrieval_task, report_analysis_task, save_analysis_task],
#     verbose=2
# )

In [11]:
app_crew = Crew(
    agents=crew_agents,
    tasks=[
        get_reads_qc_app_task, 
        get_app_params_task, 
        start_job_task, 
        make_app_cell_task,
        monitor_job_task, 
        job_completion_task, 
        report_retrieval_task, 
        report_analysis_task, 
        save_analysis_task
    ],
    verbose=2
)

2024-04-25 17:30:11,641 - 140718799765504 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [12]:
app_crew.kickoff()

 [DEBUG]: == Working Agent: Computational Biologist and Geneticist
 [INFO]: == Starting Task: Get the app id for the KBase FastQC app. Return only the app id.


> Entering new CrewAgentExecutor chain...
Action: KBase app catalog retrieval tool
Action Input: {"input": "What is the app id for the KBase FastQC app?"} 

{'query': 'What is the app id for the KBase FastQC app?', 'chat_history': '', 'result': 'Given the new context provided, the app ID for the KBase FastQC app is `kb_fastqc/runFastQC`.'}



> Finished chain.
 [DEBUG]: == [Computational Biologist and Geneticist] Task output: {'query': 'What is the app id for the KBase FastQC app?', 'chat_history': '', 'result': 'Given the new context provided, the app ID for the KBase FastQC app is `kb_fastqc/runFastQC`.'}


 [DEBUG]: == Working Agent: Computational Biologist and Geneticist
 [INFO]: == Starting Task: 
    From the given KBase app id, fetch the list of parameters needed to run it. Use the App and Job manager agent 
    for assi

'The report provides a comprehensive analysis of the quality metrics for a set of sequencing reads from the file named "rhodobacter.art.q20.int.PE.reads_74706_2_1.fwd.fastq". The key findings from the report are summarized below:\n\n- **Basic Statistics**: The sequencing file passed the basic statistics quality check. It contains a total of 193,053 sequences, all of which are of high quality with no sequences flagged as poor quality. Each sequence is 100 base pairs long, and the overall GC content is 66%.\n\n- **Per Base Sequence Quality**: The quality scores across all bases are consistently high, indicating excellent sequencing quality. The mean quality score starts at 50.4 for the first base and generally remains above 50 for most of the read, with a slight decrease towards the end. The lowest mean quality score observed is 43.47 at the 98-99th positions, but it increases again to 45.04 at the 100th position. This pattern suggests a high level of accuracy in base calling throughout 

In [7]:
app_crew.kickoff()

 [DEBUG]: == Working Agent: Computational Biologist and Geneticist
 [INFO]: == Starting Task: Get the app id for the KBase FastQC app. Return only the app id.


> Entering new CrewAgentExecutor chain...
Action: KBase app catalog retrieval tool
Action Input: {"input": "What is the app id for the KBase FastQC app?"} 

{'query': 'What is the app id for the KBase FastQC app?', 'chat_history': '', 'result': 'Given the new context provided, the app ID for the KBase FastQC app is `kb_fastqc/runFastQC`. This information directly answers the question with the specific detail needed.'}



> Finished chain.
 [DEBUG]: == [Computational Biologist and Geneticist] Task output: {'query': 'What is the app id for the KBase FastQC app?', 'chat_history': '', 'result': 'Given the new context provided, the app ID for the KBase FastQC app is `kb_fastqc/runFastQC`. This information directly answers the question with the specific detail needed.'}


 [DEBUG]: == Working Agent: Computational Biologist and Geneti

'The report from the analysis of the file "rhodobacter.art.q20.int.PE.reads_74705_2_1.fwd_fastqc.zip" using FastQC version 0.11.5 provides several key insights into the quality and characteristics of the sequencing data. Here\'s a detailed summary:\n\n- **Basic Statistics**: The analysis passed the basic statistics test. The file contains 193,053 sequences, all of which are 100 base pairs in length. Notably, no sequences were flagged as poor quality, indicating overall high-quality sequencing data. The GC content of the sequences is 66%, which is relatively high and characteristic of certain microbial genomes.\n\n- **Per Base Sequence Quality**: The quality scores across all bases are consistently high, with mean scores starting around 50.4 at the beginning of the reads and slightly decreasing towards the end. The lowest mean quality score observed is 43.47 at the 98-99th positions, but it increases again to 45.04 at the 100th position. The quality scores indicate that the sequencing d

In [15]:
job_id = "65fb44f1143dd3b9dad55cf2"
narrative_id = 72233
job_completion_task = Task(
    description=f"""Fetch the status of KBase job with id '{job_id}'.
    If it is completed without being an error, retrieve the job results.
    If the job results contain a reference to a report, with an UPA (a string with format number/number/number),
    return it. Do not add any additional text, just return only the UPA. 
    You must always use a tool when interacting with KBase services or databases. If this is delegated, make sure
    the delegated agent uses a tool when interacting with KBase.
    """,
    agent=analyst.agent,
    extra_content=job_id
)

report_retrieval_task = Task(
    description="""Using the provided UPA (a string with format number/number/number), 
    use a tool to get the report object from the Workspace service. UPAs are unique permanent addresses used to identify data objects.
    The Workspace Manager will be helpful here. Make sure the delegated agent uses a tool for interacting with KBase.
    Return the full report text. Your final answer MUST be the report text.
    """,
    agent=analyst.agent
)

report_analysis_task = Task(
    description="""Analyze the report text and derive some insight into the result. Write a brief summary, including bullet points 
    when appropriate, formatted in markdown. Your final answer MUST be the summary of the report.""",
    agent=analyst.agent
)

save_analysis_task = Task(
    description=f"""Save the analysis by adding a markdown cell to the Narrative with id {narrative_id}. The markdown text must
    be the analysis text. If not successful, say so and stop.""",
    agent=analyst.agent,
    extra_content=narrative_id
)

results_crew = Crew(
    agents=crew_agents,
    tasks=[job_completion_task, report_retrieval_task, report_analysis_task, save_analysis_task],
    verbose=2
)

2024-03-21 15:25:18,815 - 140383457423360 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [16]:
results_crew.kickoff()

[DEBUG]: Working Agent: Computational Biologist and Geneticist
[INFO]: Starting Task: Fetch the status of KBase job with id '65fb44f1143dd3b9dad55cf2'.
    If it is completed without being an error, retrieve the job results.
    If the job results contain a reference to a report, with an UPA (a string with format number/number/number),
    return it. Do not add any additional text, just return only the UPA. 
    You must always use a tool when interacting with KBase services or databases. If this is delegated, make sure
    the delegated agent uses a tool when interacting with KBase.
    


> Entering new CrewAgentExecutor chain...
Use Tool: Ask question to co-worker
Co-worker: Job and App Manager
Question: Could you fetch the status of the KBase job with id '65fb44f1143dd3b9dad55cf2' for me? If it is completed without being an error, please retrieve the job results. If the job results contain a reference to a report, with an UPA (a string with format number/number/number), could you r

'success\n```'

In [10]:
from narrative_llm_agent.kbase.clients.narrative_method_store import NarrativeMethodStore
nms = NarrativeMethodStore("https://ci.kbase.us/services/narrative_method_store/rpc")
spec = nms.get_app_spec("kb_fastqc/runFastQC")

In [11]:
spec["behavior"]

{'kb_service_url': '',
 'kb_service_name': 'kb_fastqc',
 'kb_service_version': 'b7ea69cd0fe0f62e45a8e6ea4ddeba3cba17a8d4',
 'kb_service_method': 'runFastQC',
 'kb_service_input_mapping': [{'narrative_system_variable': 'workspace',
   'target_property': 'input_ws'},
  {'input_parameter': 'input_file_ref',
   'target_property': 'input_file_ref',
   'target_type_transform': 'resolved-ref'}],
 'kb_service_output_mapping': [{'service_method_output_path': ['0',
    'report_name'],
   'target_property': 'report_name'},
  {'service_method_output_path': ['0', 'report_ref'],
   'target_property': 'report_ref'},
  {'narrative_system_variable': 'workspace',
   'target_property': 'workspace_name'}]}

In [10]:
from narrative_llm_agent.kbase.clients.execution_engine import ExecutionEngine
ee = ExecutionEngine(kbase_token, "https://ci.kbase.us/services/ee2")

In [11]:
ee.check_job("65fb44f1143dd3b9dad55cf2")

{'user': 'wjriehl',
 'authstrat': 'kbaseworkspace',
 'wsid': 72233,
 'status': 'completed',
 'updated': 1710966147960,
 'queued': 1710966001639,
 'running': 1710966010372,
 'finished': 1710966147622,
 'batch_job': False,
 'scheduler_type': 'condor',
 'scheduler_id': '61529',
 'job_input': {'wsid': 72233,
  'method': 'kb_fastqc.runFastQC',
  'params': [{'input_ws': 'wjriehl:narrative_1706895747734',
    'input_file_ref': '72233/3/1'}],
  'service_ver': 'b7ea69cd0fe0f62e45a8e6ea4ddeba3cba17a8d4',
  'app_id': 'kb_fastqc/runFastQC',
  'source_ws_objects': ['72233/3/1'],
  'requirements': {'clientgroup': 'njs',
   'cpu': 2,
   'memory': 10000,
   'disk': 100},
  'narrative_cell_info': {'run_id': 'da982d57-e369-4bfd-88e3-a5f007cb92f0',
   'tag': 'release',
   'cell_id': '1e9560a7-c107-45c8-be73-740e6f0c7bff'}},
 'job_output': {'version': '1.1',
  'result': [{'report_name': 'kb_fastqc_report_b831702e-6783-485f-a9db-ad8968fed71a',
    'report_ref': '72233/7/1'}],
  'id': '65fb44f1143dd3b9dad55

In [12]:
import requests
requests.get("https://ci.kbase.us/services/auth/api/V2/token", headers={"Authorization": kbase_token}).json()

{'type': 'Developer',
 'id': 'fa133720-5ea6-4b45-8632-6547b1ca9279',
 'expires': 1714583567055,
 'created': 1706807567055,
 'name': 'llm agency',
 'user': 'wjriehl',
 'custom': {},
 'cachefor': 300000}

In [11]:
get_reads_qc_app_task = Task(
    description = """
    I have a set of genomic reads stored in KBase that I would like to evaluate for quality control.
    Select the best KBase app for this purpose. Verify the app name with a human before proceeding.
    Using the verified app name, find and return the app id, which should have the format "<module>/<method>", and is not the app name. Return only the app id.
    """,
    agent=analyst.agent
)

In [12]:
get_app_crew = Crew(
    agents=crew_agents,
    tasks=[get_reads_qc_app_task],
    verbose=2
)

2024-03-28 02:53:05,888 - 140709533736960 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [13]:
get_app_crew.kickoff()

[DEBUG]: Working Agent: Computational Biologist and Geneticist
[INFO]: Starting Task: 
    I have a set of genomic reads stored in KBase that I would like to evaluate for quality control.
    Select the best KBase app for this purpose. Verify the app name with a human before proceeding.
    Using the verified app name, find and return the app id, which should have the format "<module>/<method>", and is not the app name. Return only the app id.
    


> Entering new CrewAgentExecutor chain...
Use Tool: KBase app catalog retrieval tool
Input: "Which KBase app is best for evaluating genomic reads for quality control?"initing doc chain with dir /home/ac.wjriehl/ke_llm/narrative_llm_agent/narrative_llm_agent/agents/vector_db_app_catalog
 

{'query': 'Which KBase app is best for evaluating genomic reads for quality control?', 'chat_history': '', 'result': 'The "Reads Quality Control with FastQC - v1.0.1" KBase app is best for evaluating genomic reads for quality control. It provides a modula

 yes


 

yes

Use Tool: KBase app catalog retrieval tool
Input: "What is the app id for the 'Reads Quality Control with FastQC - v1.0.1' KBase app?"initing doc chain with dir /home/ac.wjriehl/ke_llm/narrative_llm_agent/narrative_llm_agent/agents/vector_db_app_catalog
 

{'query': "What is the app id for the 'Reads Quality Control with FastQC - v1.0.1' KBase app?", 'chat_history': '', 'result': "The context provided still does not include specific details about the app id for the 'Reads Quality Control with FastQC - v1.0.1' KBase app."}

Use Tool: Ask question to co-worker
Co-worker: Workspace Manager
Question: "Can you provide the app id for the 'Reads Quality Control with FastQC - v1.0.1' KBase app?"
Context: "I have confirmed with a human that the 'Reads Quality Control with FastQC - v1.0.1' KBase app is the best for evaluating genomic reads for quality control. However, I need the app id, which should have the format '<module>/<method>', and is not the app name. I tried using the KBase ap

"I'm sorry, but I was unable to retrieve the app id for the 'Reads Quality Control with FastQC - v1.0.1' KBase app. The tool I used encountered an error indicating that the repository 'ReadsQualityControl' wasn't registered. I recommend reaching out to KBase support for further assistance."